In [28]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import numpy as np

In [1]:
import sys
path_module = r'C:\Users\je7560\Downloads\matheus\kinematics-robotics\src\kinematicsrobotics'
#path_module = r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics\src\kinematicsrobotics'
sys.path.append(path_module)
import datahandler

## Base de dados

In [25]:
path_data = r'src\data\ready\dataset-radius-0.5cm.csv'
#path_project = r'C:\Users\mathe\OneDrive\Graduação - UFC\Engenharia da Computação\TCC\Códigos e implementações\V.2\kinematics-robotics'
path_project = r'C:\Users\je7560\Downloads\matheus\kinematics-robotics'
ext = datahandler.extract(path_project)

dataset = ext.dataframe(path_data)


### Atributos e saídas

#### Análise dos dados

In [26]:
print(dataset.columns)

Index(['theta_1', 'theta_2', 'theta_3', 'theta_4', 'theta_5', 'p_x', 'p_y',
       'p_z', 'roll', 'pich', 'yaw'],
      dtype='object')


In [27]:
from numpy import array
def partition(dataset,axis):
    return array(dataset.iloc[:,axis[0]:axis[1]])


y = partition(dataset,[0,4])
x = partition(dataset,[5,11])

In [28]:
print(f"Tamanho do dataset: {y.shape}")

Tamanho do dataset: (15324, 4)


In [29]:
y[0:3]

array([[ 0,  0,  0,  0],
       [ 0,  0, 12,  0],
       [ 0,  0, 12, 12]], dtype=int64)

In [30]:
x[0:3]

array([[ 5.40000000e+01, -1.10218212e-15,  1.00000000e+01,
         2.35619449e+00, -1.57079633e+00,  7.85398163e-01],
       [ 5.32133136e+01, -2.47712141e-15,  2.51517913e+00,
         3.14159265e+00, -1.36135682e+00,  2.75203954e-16],
       [ 5.36066568e+01, -2.24796486e-15,  6.25758957e+00,
         2.33481267e+00, -1.57079633e+00,  8.06779988e-01]])

## Pré-processamento de dados

In [24]:
size_train = 0.7
size_val = 0.2
size_test = 0.1

In [31]:
from sklearn.model_selection import train_test_split
def split(x, y, test_size):
    return train_test_split(x, y, test_size=test_size)

### Divisão treino e teste

In [32]:
x_train, x_test, y_train, y_test = split(x,y,0.1)

#### informações

In [33]:
print(f"{len(x_train)}")
print(f"{len(x_test)}")

13791
1533


### Normalização dos dados

In [10]:
from sklearn.preprocessing import StandardScaler
from numpy import mean,std

def normalize(data):
    scaler = StandardScaler()
    means = mean(data,axis=0)
    stds = std(data,axis=0)
    data = scaler.fit_transform(data)
    return data, means,stds



## Modelo

In [18]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in ss.split(x_train, y_train):
    print("Índices de Treino:", len(train_index))
    print("Índices de Teste:", len(test_index))
    print("----------------------")

Índices de Treino: 11032
Índices de Teste: 2759
----------------------
Índices de Treino: 11032
Índices de Teste: 2759
----------------------
Índices de Treino: 11032
Índices de Teste: 2759
----------------------


In [ ]:
from sklearn.neural_network import MLPRegressor
def modelo(hidden):
    func_act = 'tanh'
    model = MLPRegressor(hidden_layer_sizes=hidden,
                        activation=func_act)

In [126]:
# Definir os parâmetros
taxa_aprendizado = 0.01
numero_epocas = 1000
erro_adm = 0.01
fator_momento = 0.9
validacao = 0.1

In [129]:
model = MLPRegressor(hidden_layer_sizes=(100, 200), #validação
                    activation='tanh',  # Função de ativação da camada oculta
                    random_state=42)

## Seleção de modelos

In [24]:
def grid_search(X_train, y_train, param_grid, model):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    #.ravel()

    # Avaliação do melhor hiperparâmetro
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)

    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print(f"\nMSE nos dados completos: {mse}")
    print(f"R² nos dados completos: {r2}")

    # Extrair os resultados da validação cruzada
    resultados = pd.DataFrame(grid_search.cv_results_)

    colunas_interesse = ['param_hidden_layer_sizes', 'mean_test_score', 'std_test_score']
    resultados_interesse = resultados[colunas_interesse]

    print("Avaliação do MSE para cada hiperparâmetro testado:")
    print(resultados_interesse)
    return resultados

### Modelo

In [130]:
# Treinando o modelo
model.fit(Xtrain, Ytrain)

# Fazendo previsões
y_pred = model.predict(Xtrain)

# Avaliando o modelo
mse = mean_squared_error(Ytrain, y_pred)
print(mse)
#print(f"{Ytrain}  {y_pred}")

0.008671714073118121


## Avaliação de modelos


Métricas de regressão (MSE, RMSE, R²)
Validação cruzada
Curvas de aprendizado
Matrizes de confusão.
Grid search e busca aleatória de hiperparâmetros.

In [ ]:
from sklearn.metrics import mean_squared_error

def predict_mse(model,X_test,Y_test):
    y_predic = model.predict(X_test)
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')